In [1]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten


from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re

In [2]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
# dataset = dataset.loc[:,cols]
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class

dataset.sample(10)

,Area,Perimeter,Major_Axis,Minor_Axis,Eccentricity,Eqdiasq,Solidity,Convex_Area,Extent,Aspect_Ratio,...,StdDev_RR,StdDev_RG,StdDev_RB,Skew_RR,Skew_RG,Skew_RB,Kurtosis_RR,Kurtosis_RG,Kurtosis_RB,Class
1216,72405,1150.8480,470.9488,198.5195,0.9068,303.6263,0.9654,74997,0.7682,2.3723,...,20.7490,22.0774,25.1499,-0.7005,-0.3783,0.1049,2.9508,2.7279,2.7598,Kirmizi_Pistachio
701,78536,2738.6531,453.4380,252.2755,0.8309,316.2201,0.8772,89533,0.7092,1.7974,...,25.8387,28.5591,25.0407,-0.6623,-0.3644,-0.3421,2.4653,2.3024,2.3167,Kirmizi_Pistachio
1132,72714,1151.3250,432.3479,217.9682,0.8636,304.2735,0.9809,74133,0.7431,1.9835,...,24.6671,26.5102,29.1979,-1.4967,-1.2236,-0.9223,4.3975,3.6851,2.9670,Kirmizi_Pistachio
1242,82217,1127.7930,433.8209,248.6817,0.8194,323.5459,0.9581,85812,0.7423,1.7445,...,22.8421,24.2989,19.2319,-0.8524,-0.9491,-0.8562,2.7947,2.9686,2.9200,Siirt_Pistachio
581,72682,1228.4550,483.3133,194.6325,0.9153,304.2065,0.9681,75075,0.7105,2.4832,...,14.9318,15.1270,16.0761,-0.3506,-0.1983,0.0439,3.4227,3.4293,4.7344,Kirmizi_Pistachio
1948,92827,1778.8750,470.9737,257.7242,0.8370,343.7892,0.9451,98219,0.7462,1.8274,...,18.3209,18.9946,20.0495,-0.9675,-0.7858,-0.5884,3.5007,3.1006,2.8007,Siirt_Pistachio
71,57539,1852.1949,415.5954,218.6565,0.8504,270.6676,0.7909,72752,0.6211,1.9007,...,20.9036,20.9947,18.9361,-0.3635,-0.3427,-0.4945,2.2780,2.3152,2.9684,Kirmizi_Pistachio
461,72245,1148.2860,436.4400,221.4601,0.8617,303.2906,0.9496,76079,0.7413,1.9707,...,23.5816,23.0104,20.1304,-0.7414,-0.7705,-0.7401,2.3595,2.4538,2.6496,Kirmizi_Pistachio
987,83711,1903.6750,480.3589,256.2626,0.8458,326.4723,0.8505,98431,0.7089,1.8745,...,24.2100,25.7157,25.7350,-0.3267,-0.1776,0.2028,2.3101,2.2764,2.3999,Kirmizi_Pistachio
1528,88616,1244.3020,440.4996,259.9724,0.8073,335.9009,0.9661,91728,0.7618,1.6944,...,20.7828,22.1591,22.8526,-0.7152,-0.6702,-0.3910,2.7049,2.6018,2.6033,Siirt_Pistachio


In [3]:
dataset = shuffle(dataset)
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [4]:
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class 


In [5]:
#X.dtype = 'int64'
# Y.dtype = 'int64'
# X = X.astype(int)

# X = X.astype(int)

In [6]:
print(X.shape)
print(Y.shape)

(2130, 28)
(2130, 2)


In [7]:
import os,sys
import shutil

shutil.rmtree('models')  
os.mkdir('models')

In [8]:
# import tensorflow as tf
# def metric_precision(y_true,y_pred):
#     TP=tf.reduce_sum(y_true/tf.round(y_pred))
#     TN=tf.reduce_sum((1-y_true)(1-tf.round(y_pred)))
#     FP=tf.reduce_sum((1-y_true)tf.round(y_pred))
#     FN=tf.reduce_sum(y_true(1-tf.round(y_pred)))
#     precision=TP/(TP+FP)
#     return precision

In [9]:
def accsorce(Y_test,yhat):
    sorce = 0
    for i in range(len(Y_test)):
        sorce += roc_auc_score(Y_test[i], yhat[i])

    return sorce/len(Y_test)

In [10]:

#定义模型
import random
sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)

lstm_layers = [1,2,3,4,5]
dense_layers = [1,2,3,4,5,6]
units = [16,32,64,128]
dropout = [0.05,0.1,0.15,0.25]
Batch_size = [32,64,128]
optimizer = ['adam',sgd]
for the_batch_size in Batch_size:
    for the_dropout in dropout:
        for the_optimizer in optimizer:
            for the_dense_layers in dense_layers:
                for the_lstm_layers in lstm_layers:
                    for the_units in units:
                        sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
                        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=40)
                        X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
                        X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
                        model = Sequential()
                        # model.build(input_shape=(277,277,2))
                        #print(model.summary())
                        #model.add(SpatialDropout1D(0.2))
                        model.add(LSTM(the_units ,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences = True))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        # #第二层
                        for i in range(the_lstm_layers):
                            model.add(LSTM(the_units,return_sequences=True))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())

                        model.add(LSTM(the_units))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        #全连接层
                        for i in range(the_dense_layers):
                            model.add(Dense(the_dense_layers,activation='relu'))
                            model.add(Dropout(the_dropout))
                            
                        # model.add(Flatten()) 
                        
                        
                        model.add(Dense(2, activation='softmax'))
                        
                        #sgd = SGD(learning_rate=0.01, momentum=0.9 , decay=0.1, nesterov=False)
                        
                        # learning_rate = 0.1
                        # decay = 0.001
                        # epochs = 50
                        # batch_size = 64
                        
                        
                        model.compile(  loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                                        optimizer=the_optimizer, metrics=['categorical_accuracy'])
                        print(model.summary())

                        epochs = 100
                        batch_size = the_batch_size
                        if(the_optimizer == sgd):
                            the_optimizer = 'sgd'
                        filepath = './models/{categorical_accuracy:.4f}_{epoch:02d}_'+f'dropout_{the_dropout}_batch_size_{the_batch_size}_optimizer_{the_optimizer}_dense_layers_{the_dense_layers}_lstm_layers_{the_lstm_layers}_unit_{the_units}.h5'
                        checkpoint = ModelCheckpoint(
                                            filepath=filepath,
                                            save_weights_only=False,
                                            monitor='categorical_accuracy',
                                            mode='max',
                                            save_best_only=True)
                        history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1,
                                            #callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
                                            callbacks=checkpoint)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 16)             2880      
_________________________________________________________________
dropout (Dropout)            (None, 1, 16)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 16)             64        
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 16)             2112      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 16)             0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 16)             64        
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                2

In [ ]:
#model.save('mymodel.h5')

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/0.9557_78_dropout_0.05pbatch_size_32optimizer_adamdense_layers_1_lstm_layers_2_unit_128.h5')

In [ ]:
dataset = pd.read_csv('test.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
# dataset = dataset.loc[:,cols]
dataset.sample(10)
dataset = shuffle(dataset)
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#进行预测 make a prediction
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.90, random_state=random.randint(10,100))
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
# print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
yhat = best_model.predict(X_test)
# print(yhat.shape)
print(yhat)
print(Y_test)
sorce = 0
for i in range(len(Y_test)):
    sorce += roc_auc_score(Y_test[i], yhat[i])

print('ACC:',sorce/len(Y_test))
#拆分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=random.randint(10,100))
# X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [ ]:
print(Y_test.shape)
print(yhat.shape)
# Y_test = Y_test.reshape(430)
# yhat = yhat.reshape(430)
# Y_test = list(Y_test)
# yhat = list(yhat)

In [ ]:
# print(yhat)